In [5]:
# MENGIMPOR SEMUA LIBRARY YANG DIPERLUKAN UNTUK BAB 6
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

# Impor untuk Decision Trees
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz # Untuk visualisasi

# Impor untuk data non-linear dan perbandingan
from sklearn.datasets import make_moons
from sklearn.inspection import DecisionBoundaryDisplay

# Mengatur default plotting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Mengatur path untuk menyimpan gambar
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

print("--- BAB 6: DECISION TREES (POHON KEPUTUSAN) ---")
print("Semua library telah diimpor.")

# 1. MELATIH DAN MEMVISUALISASIKAN DECISION TREE (KLASIFIKASI)
print("\n[1. Melatih dan Memvisualisasikan (Klasifikasi)]")

# Teori: Decision Trees adalah model 'white box' yang intuitif.
# Mereka adalah serangkaian pertanyaan "jika-maka" yang dipelajari dari data.

# Menggunakan dataset Iris
iris = load_iris()
X_iris = iris.data[:, 2:] # Fitur: petal length (cm) dan petal width (cm)
y_iris = iris.target

# Melatih DecisionTreeClassifier
# Kita batasi kedalaman pohon (max_depth=2) untuk mencegah overfitting
# dan agar mudah divisualisasikan. Ini adalah bentuk REGULARISASI.
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_iris, y_iris)

print("DecisionTreeClassifier (max_depth=2) telah dilatih pada data Iris.")

# Memvisualisasikan pohon yang sudah dilatih
# Ini menghasilkan file .dot yang bisa dikonversi ke PNG/PDF
# menggunakan alat 'dot' dari Graphviz.
# (Anda harus menginstal Graphviz secara terpisah di sistem Anda untuk menjalankan 'dot')
print("Mengekspor visualisasi pohon ke 'iris_tree.dot'...")
export_graphviz(
        tree_clf,
        out_file=os.path.join(IMAGES_PATH, "iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

print(f"File 'iris_tree.dot' telah disimpan di {IMAGES_PATH}")
print("  Untuk melihat, jalankan di terminal Anda:")
print(f"  dot -Tpng {IMAGES_PATH}/iris_tree.dot -o {IMAGES_PATH}/iris_tree.png")


# 2. MEMBUAT PREDIKSI DAN ESTIMASI PROBABILITAS
print("\n[2. Membuat Prediksi dan Estimasi Probabilitas]")
# Teori: Prediksi dibuat dengan 'menelusuri' pohon dari root (akar)
# hingga ke leaf node (daun) berdasarkan fitur instance.

# Penjelasan file .dot:
# - samples: Berapa banyak instance latih yang masuk ke node ini.
# - value: [jml kelas 0, jml kelas 1, jml kelas 2] di node ini.
# - gini: Gini Impurity (ukuran kemurnian). 0 = murni (semua instance 1 kelas).
# - class: Kelas yang diprediksi di node ini (berdasarkan 'value' mayoritas).

# Contoh prediksi: petal length=5cm, petal width=1.5cm
# 1. (Root): petal length <= 2.45? (5 <= 2.45 adalah False) -> Pergi ke node Kanan.
# 2. (Depth 1, Kanan): petal width <= 1.75? (1.5 <= 1.75 adalah True) -> Pergi ke node Kiri.
# 3. (Depth 2, Kiri): Ini adalah leaf node. Prediksi = 'versicolor' (value=[0, 49, 5])

print(f"Prediksi untuk [petal length=5, petal width=1.5]:")
print(f"  Kelas (prediksi): {tree_clf.predict([[5, 1.5]])}") # Output: [1] (versicolor)

# Teori: Probabilitas adalah rasio instance kelas di leaf node tempat instance tersebut jatuh.
# Node [0, 49, 5] memiliki total 54 sampel.
# P(setosa) = 0/54 = 0%
# P(versicolor) = 49/54 = ~90.7%
# P(virginica) = 5/54 = ~9.3%
print(f"  Probabilitas kelas: {tree_clf.predict_proba([[5, 1.5]])}")


# 3. REGULARISASI HYPERPARAMETERS
print("\n[3. Regularisasi Hyperparameters]")
# Teori: Jika tidak dibatasi, Decision Trees akan terus membelah
# sampai setiap daun murni. Ini menyebabkan OVERFITTING parah.
# Kita menggunakan hyperparameter untuk membatasinya (meregularisasi).

# Hyperparameter regularisasi utama:
# - max_depth: Kedalaman maksimum pohon.
# - min_samples_split: Jml minimum instance agar node bisa dipecah.
# - min_samples_leaf: Jml minimum instance yang harus ada di leaf node.
# - max_features: Jml maksimum fitur yang dievaluasi untuk split.
# Mengubah hyperparameter ini (menaikkan min_*, menurunkan max_*)
# akan meningkatkan regularisasi dan mengurangi overfitting.

# Contoh: Melatih pada dataset moons (non-linear)
Xm, ym = make_moons(n_samples=100, noise=0.25, random_state=53)

# Model 1: Tanpa regularisasi (default)
tree_clf1 = DecisionTreeClassifier(random_state=42)
tree_clf1.fit(Xm, ym)

# Model 2: Dengan regularisasi (min_samples_leaf=4)
tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
tree_clf2.fit(Xm, ym)

print("Telah melatih 2 model (overfit vs regularisasi) pada dataset moons.")

# Plot decision boundaries (di-comment agar tidak auto-tampil)
# plt.figure(figsize=(11, 4))
# plt.subplot(121)
# DecisionBoundaryDisplay.from_estimator(tree_clf1, Xm, alpha=0.4, response_method="predict")
# plt.plot(Xm[:, 0][ym==0], Xm[:, 1][ym==0], "bs")
# plt.plot(Xm[:, 0][ym==1], Xm[:, 1][ym==1], "g^")
# plt.title("Tanpa Regularisasi (Overfitting)")
# plt.subplot(122)
# DecisionBoundaryDisplay.from_estimator(tree_clf2, Xm, alpha=0.4, response_method="predict")
# plt.plot(Xm[:, 0][ym==0], Xm[:, 1][ym==0], "bs")
# plt.plot(Xm[:, 0][ym==1], Xm[:, 1][ym==1], "g^")
# plt.title("Dengan Regularisasi (min_samples_leaf=4)")
# plt.show()
# Model 2 (kanan) terlihat memiliki generalisasi yang lebih baik.

# 4. REGRESI DENGAN DECISION TREES
print("\n[4. Regresi dengan Decision Trees]")
# Teori: Pohon juga bisa dipakai untuk regresi.
# Perbedaannya adalah:
# 1. Kriteria split: Alih-alih meminimalkan Gini/Entropy (impurity),
#    algoritma CART meminimalkan MSE (Mean Squared Error).
# 2. Prediksi: Alih-alih memprediksi 'kelas' di leaf node, pohon
#    memprediksi 'nilai' (yaitu nilai rata-rata dari semua instance di leaf tsb).

# Membuat data regresi kuadratik palsu
np.random.seed(42)
m_reg = 200
X_reg = np.random.rand(m_reg, 1)
y_reg = 4 * (X_reg - 0.5) ** 2 + np.random.randn(m_reg, 1) / 10 # y = 4(x-0.5)^2 + noise

# Melatih DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X_reg, y_reg)

tree_reg2 = DecisionTreeRegressor(max_depth=3, random_state=42)
tree_reg2.fit(X_reg, y_reg)

print("DecisionTreeRegressor (max_depth=2 dan 3) telah dilatih.")

# Visualisasi hasil regresi (di-comment)
# X_new_reg = np.linspace(0, 1, 500).reshape(-1, 1)
# y_pred_reg1 = tree_reg1.predict(X_new_reg)
# y_pred_reg2 = tree_reg2.predict(X_new_reg)
#
# plt.figure(figsize=(11, 4))
# plt.subplot(121)
# plt.plot(X_reg, y_reg, "b.")
# plt.plot(X_new_reg, y_pred_reg1, "r-", linewidth=2, label=r"$\hat{y}$ (max_depth=2)")
# plt.axis([0, 1, -0.2, 1.1])
# plt.xlabel("$x_1$")
# plt.ylabel("$y$")
# plt.legend()
#
# plt.subplot(122)
# plt.plot(X_reg, y_reg, "b.")
# plt.plot(X_new_reg, y_pred_reg2, "r-", linewidth=2, label=r"$\hat{y}$ (max_depth=3)")
# plt.axis([0, 1, -0.2, 1.1])
# plt.xlabel("$x_1$")
# plt.legend()
# plt.show()
# Plot menunjukkan bahwa prediksi adalah nilai rata-rata konstan di dalam setiap leaf.
# Model regresi pohon (tanpa regularisasi) juga akan overfit.
# tree_reg_overfit = DecisionTreeRegressor(random_state=42)
# tree_reg_overfit.fit(X_reg, y_reg) # Akan sangat overfit

# 5. KETIDAKSTABILAN DECISION TREES
print("\n[5. Kelemahan: Ketidakstabilan]")
# Teori: Kelemahan utama Decision Trees adalah mereka SANGAT sensitif
# terhadap variasi kecil pada data latih dan rotasi data.

# Contoh sensitivitas terhadap rotasi data
np.random.seed(6)
Xs = np.random.rand(100, 2) - 0.5
ys = (Xs[:, 0] > 0).astype(np.float32) * 2

angle = np.pi / 4  # Rotasi 45 derajat
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xsr = Xs.dot(rotation_matrix)

# Latih pohon pada data asli dan data yang dirotasi
tree_clf_s = DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(Xs, ys)
tree_clf_sr = DecisionTreeClassifier(random_state=42)
tree_clf_sr.fit(Xsr, ys)

print("Melatih 2 pohon: 1 pada data asli, 1 pada data yang dirotasi 45 derajat.")

# Plot perbandingan (di-comment)
# plt.figure(figsize=(11, 4))
# plt.subplot(121)
# DecisionBoundaryDisplay.from_estimator(tree_clf_s, Xs, alpha=0.4, response_method="predict")
# plt.plot(Xs[:, 0][ys==0], Xs[:, 1][ys==0], "bs")
# plt.plot(Xs[:, 0][ys==1], Xs[:, 1][ys==1], "g^")
# plt.title("Data Asli (Batas Ortogonal)")
#
# plt.subplot(122)
# DecisionBoundaryDisplay.from_estimator(tree_clf_sr, Xsr, alpha=0.4, response_method="predict")
# plt.plot(Xsr[:, 0][ys==0], Xsr[:, 1][ys==0], "bs")
# plt.plot(Xsr[:, 0][ys==1], Xsr[:, 1][ys==1], "g^")
# plt.title("Data yang Dirotasi 45° (Batas Rumit)")
# plt.show()

# Hasil: Pohon di data asli (kiri) punya batas keputusan sederhana.
# Pohon di data yang dirotasi (kanan) punya batas keputusan yang
# sangat rumit dan tidak perlu (overfitting).
# Ini adalah kelemahan utama. Random Forest (Bab 7) akan memperbaiki ini.

print("\n--- Selesai Bab 6 ---")

--- BAB 6: DECISION TREES (POHON KEPUTUSAN) ---
Semua library telah diimpor.

[1. Melatih dan Memvisualisasikan (Klasifikasi)]
DecisionTreeClassifier (max_depth=2) telah dilatih pada data Iris.
Mengekspor visualisasi pohon ke 'iris_tree.dot'...
File 'iris_tree.dot' telah disimpan di ./images/decision_trees
  Untuk melihat, jalankan di terminal Anda:
  dot -Tpng ./images/decision_trees/iris_tree.dot -o ./images/decision_trees/iris_tree.png

[2. Membuat Prediksi dan Estimasi Probabilitas]
Prediksi untuk [petal length=5, petal width=1.5]:
  Kelas (prediksi): [1]
  Probabilitas kelas: [[0.         0.90740741 0.09259259]]

[3. Regularisasi Hyperparameters]
Telah melatih 2 model (overfit vs regularisasi) pada dataset moons.

[4. Regresi dengan Decision Trees]
DecisionTreeRegressor (max_depth=2 dan 3) telah dilatih.

[5. Kelemahan: Ketidakstabilan]
Melatih 2 pohon: 1 pada data asli, 1 pada data yang dirotasi 45 derajat.

--- Selesai Bab 6 ---
